# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES(x0=np.random.random(problem.n_var))

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")


Best solution found: 
X = [0.50000008 0.49999995 0.5        0.49999998 0.49999993 0.50000001
 0.50000004 0.50000002 0.5        0.50000004]
F = [1.79526187e-14]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  1.150133657 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.672881389 |  0.097913978 |  0.09510 |  0.10104 |  1.21755
    3 |      21 |  0.672881389 |  0.098430988 |  0.09326 |  0.10374 |  1.32524
    4 |      31 |  0.467597676 |  0.105834033 |  0.09823 |  0.11826 |  1.50111
    5 |      41 |  0.368776034 |  0.115319569 |  0.10545 |  0.13158 |  1.69164
    6 |      51 |  0.295774169 |  0.134776025 |  0.12187 |  0.15404 |  1.80065
    7 |      61 |  0.238464130 |  0.147693685 |  0.13139 |  0.17544 |  1.83073
    8 |      71 |  0.223148174 |  0.157851984 |  0.13734 |  0.18722 |  1.88988
    9 |      81 |  0.123746427 |  0.163995207 |  0.14164 |  0.19367 |  1.94769
   10 |      91 |  0.103185622 |  0.181337207 |  0.15649 |  0.22722 |  1.99108
Best solution found: 
X = [0.59990517 0.41086129 0.46979739 0.3397611  0.51778807 0.55426864
 0.49978018 0.59904646 0.32619809 0.62

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  1.150133657 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.836704410 |  0.101572567 |  0.09847 |  0.10770 |  1.26745
    3 |      21 |  0.704695241 |  0.101321325 |  0.09512 |  0.10792 |  1.36035
    4 |      31 |  0.672945202 |  0.102647730 |  0.09620 |  0.10956 |  1.40331
    5 |      41 |  0.566218849 |  0.110520194 |  0.10003 |  0.12027 |  1.57366
    6 |      51 |  0.502130332 |  0.124663201 |  0.11324 |  0.13814 |  1.77382
Best solution found: 
X = [0.60410202 0.89946133 0.92881354 0.70417412 0.46730508 0.4367235
 0.45083547 0.45842728 0.80300223 0.57160279]
F = [0.50213033]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [ 2.58289150e-07 -4.22497009e-07]
F = [4.86508611e-11]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-9.94959595e-01 -4.89908944e-07]
F = [0.99495906]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API